In [1]:
from MAG_network import CitationNetwork
import os 
from matplotlib import pyplot as plt
import pandas as pd
import findspark
import MAGspark 

# set environment variables
os.environ["SPARK_LOCAL_DIRS"] = "/home/laal/MAG/TMP"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-1.8.0-openjdk-1.8.0.242.b08-0.el7_7.x86_64"
os.environ['SPARK_HOME'] = "/home/laal/MAG/spark-3.0.2-bin-hadoop2.7"

In [2]:
mag, spark = MAGspark.get_mag_with_cluster_connection(jobid=40252, memory_per_executor=14000,
                                                      data_folderpath="/home/laal/MAG/DATA/")

['NAME STATE JOBID', 'ETL_wikidata_dump RUNNING 40170', 'ETL_wikidata_dump RUNNING 40201', 'generatedata RUNNING 40250', 'sparkcluster RUNNING 40252', '2.train.nmt.5 RUNNING 40188', '2.train.nmt.4 RUNNING 40187', 'muzero-gpu-job RUNNING 39988', 'train-network-incv RUNNING 40251', 'jupyter RUNNING 40253', '2.train.nmt.7 RUNNING 40190', '2.train.nmt.6 RUNNING 40189', '2.train.nmt.0 RUNNING 40183', '2.train.nmt.1 RUNNING 40184', '2.train.nmt.9 RUNNING 40192', '2.train.nmt.8 RUNNING 40191', '2.train.nmt.2 RUNNING 40185', '2.train.nmt.3 RUNNING 40186', '']


In [3]:
sociology = CitationNetwork(mag, fos_id=144024400, fos_name="Sociology", root_data_folder="/home/laal/MAG/DATA")

In [4]:
sociology.check_references_and_citations()

Paper references and citations available for Sociology


In [5]:
sociology.save_author_network("SimpleWeightSociology2010", mindate='1800-01-01', maxdate='2010-12-31')
network_df = sociology.load_author_author_network("SimpleWeightSociology2010")

Network exists at /home/laal/MAG/DATA/NETWORKS/SimpleWeightSociology2010.txt. Use overwrite to replace


In [6]:
network_df, nodelist = sociology.nodelist_and_edge_count()

The network has 6233863 edges
The network has 635826 nodes


In [7]:
graph, node_mapping, eweight = sociology.build_graph()
sociology.compute_centralities(graph, node_mapping, eweight, "", pr_damping=0.85)
sociology.append_gender_and_macrank()

Initiating PageRank
Initiating degree measures
Finished centrality computations
Centrality CSV saved to /home/laal/MAG/DATA/NETWORKS/SimpleWeightSociology2010Centrality.csv
Gendered centrality measures saved to /home/laal/MAG/DATA/NETWORKS/SimpleWeightSociology2010CentralityGendered.csv


In [1]:
spark.stop()

NameError: name 'spark' is not defined

### Rerun networks

In [3]:
id_names = [(185592680, 'Chemistry'), (15744967, 'Psychology'), (33923547, 'Mathematics'), (162324750, 'Economics')]

for fos_id, name in id_names:
    print("Starting network extraction for {}".format(name))
    
    network = CitationNetwork(mag, fos_id=fos_id, fos_name=name, root_data_folder="/home/laal/MAG/DATA")
    network.check_references_and_citations()
    network.save_author_network("SimpleWeight{}2020".format(name), mindate='1800-01-01', maxdate='2020-12-31')
    network.load_author_author_network("SimpleWeight{}2020".format(name))
    network.nodelist_and_edge_count()    
    print("\n----------------------------\n")

Starting network extraction for Chemistry
Paper references and citations available for Chemistry
Network exists at /home/laal/MAG/DATA/NETWORKS/SimpleWeightChemistry2020.txt. Use overwrite to replace
The network has 1361642334 edges
The network has 15845298 nodes
Gender distribution: -1    11910223
 1     2580695
 0     1354380
Name: Gender, dtype: int64

----------------------------

Starting network extraction for Psychology
Paper references and citations available for Psychology
Network exists at /home/laal/MAG/DATA/NETWORKS/SimpleWeightPsychology2020.txt. Use overwrite to replace
The network has 419209380 edges
The network has 4088513 nodes
Gender distribution: -1    2464853
 0     814668
 1     808992
Name: Gender, dtype: int64

----------------------------

Starting network extraction for Mathematics
Paper references and citations available for Mathematics
Network exists at /home/laal/MAG/DATA/NETWORKS/SimpleWeightMathematics2020.txt. Use overwrite to replace
The network has 9447

In [4]:
id_names = [(15744967, 'Psychology'), (33923547, 'Mathematics'), (162324750, 'Economics')]
id_names = [(185592680, 'Chemistry')]

for fos_id, name in id_names:
    print("Starting network extraction for {}".format(name))
    
    network = CitationNetwork(mag, fos_id=fos_id, fos_name=name, root_data_folder="/home/laal/MAG/DATA")
    network.check_references_and_citations()
    network.save_author_network("SimpleWeight{}2020".format(name), mindate='1800-01-01', maxdate='2020-12-31')
    network.load_author_author_network("SimpleWeight{}2020".format(name))
    
    # graph, node_mapping, eweight = network.build_graph()
    # network.compute_centralities(graph, node_mapping, eweight, "", pr_damping=0.85)
    network.append_gender_and_macrank()
    
    # network.nodelist_and_edge_count()    
    print("\n----------------------------\n")

Starting network extraction for Chemistry
Paper references and citations available for Chemistry
Network exists at /home/laal/MAG/DATA/NETWORKS/SimpleWeightChemistry2020.txt. Use overwrite to replace
Gendered centrality measures saved to /home/laal/MAG/DATA/NETWORKS/SimpleWeightChemistry2020CentralityGendered.csv

----------------------------

